# proof of concept 

## Predict sang genre, givet sangtekst vha. Naive Bayes

### 1. generel opsætning

In [1]:
# Imports
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
from langdetect import detect 

import pandas as pd

import os
import random
import zipfile


In [2]:
# functions
def _detect_lan(lyrics):
    try:
        return detect(lyrics)
    except:
        return 'unknown'
    
def _create_word_features(words):
    return dict( [ (word, True) for word in words] )


### 2. opret DataFrame og rens datasæt

In [3]:
# Load dataset
# import raw dataset
csv_file = './data/lyrics.csv'

if not os.path.isfile(csv_file):
    with zipfile.ZipFile(f'{csv_file}.zip', 'r') as zip_ref:
        zip_ref.extractall('./data')
        
df = pd.read_csv(csv_file)
rows, _= df.shape
print('datasættet i sin "raw" form indeholder:')
print(f'{rows} antal rækker')


datasættet i sin "raw" form indeholder:
362237 antal rækker


### udvind features og tilføj til datasæt
Hvis det er første gang dette step køres vær opmærksom på, at der laves en tung sproglig analyse af sangteksterne. Det vil derfor resulterer i, siden der er MANGE rækker data, at det kan tage 20+ minutter at oversætte alle sangtekster, hvis hele datasættet benyttes. Det kan derfor anbefales, at man tager en mindre sample af datasættet.

Hvis dette step ER kørt, så burde der være gemt en .pkl i ./data som gør, at feature genereringen kan springes over.

Der skal ligges mærke til, at når der udtages sample af datasættet, så ligges de enkelte udvalgte rækker tilbage. Dette er en IKKE en rigtig måde at gøre det på, da der så kan fremkomme duplikationer af rækker. Det er derimod for, at lave et "proof of concept" med mulighed for, at benytte alle genre i træning af modellen.

In [4]:
FEATURE_DATASET_FILE = './data/feature_dataset.pkl'
SAMPLE_SIZE = 1000 # <-- ændre denne hvis et anden størrelse datasæt ønskes

if not os.path.isfile(FEATURE_DATASET_FILE):
    df_cp = df.copy() # <-- kopi af importeret datasæt

    ###
    # Oprydning i raw datasæt
    ###
    # fjern rækker med nan værdier
    df_cp = df_cp.dropna()

    # fjern uønskede genre (Not Available & Other)
    df_cp = df_cp[(df_cp.genre != 'Not Available') & (df_cp.genre != 'Other')]

    ###
    # Udtaget stikprøve af datasæt
    ###
    grouped = df_cp.groupby('genre')
    grouped = grouped.apply(lambda x: x.sample(n=SAMPLE_SIZE, replace=True))
    df_cp = grouped.reset_index(drop=True)
    
    ###
    # Tilføj features til datasæt
    ###
    # bestem sprog for sangtekst
    print('detect language...', end='')
    df_cp['lan'] = df_cp.lyrics.apply(_detect_lan)
    df_cp = df_cp[df_cp.lan == 'en'] # kun benyt engelske sangtekster
    print('DONE')
    
    # find unikke ord i sangtekst
    print('generate unqiue-words feature...', end='')
    df_cp['words'] = df_cp.apply(lambda p: (_create_word_features(word_tokenize(p.lyrics)),p.genre), axis=1)
    print('DONE')
    
    # fjern col 'index'
    df_cp.drop(['index'], axis=1, inplace=True)
    
    print('saving feature dataset to pickle...', end='')
    df_cp.to_pickle(FEATURE_DATASET_FILE)
    print('DONE')
else:
    print('reading feature dataset from pickle...', end='')
    df_cp = pd.read_pickle(FEATURE_DATASET_FILE)
    print('DONE')


detect language...DONE
generate unqiue-words feature...DONE
saving feature dataset to pickle...DONE


In [52]:
# serialize to pickle
df_cp.to_pickle("./data/clean_with_lan.pkl")


### 4. klargør ord-feature til Naive Bayes
Denne del kan tage +10 min. Sørg derfor IKKE, at overskrive `clean_with_lan_and_words.pkl` med andet! 

Hvis du har adgang til `clean_with_lan_and_words.pkl` spring dette step over. Det kan ikke svare sig, at oprette datasættet igen. 

Naive Bayes tager imod en tuple med følgende syntaks `(word_list, stop_word)`, derfor giver det mening af oprette en feature til datasættet som indeholder netop dette.

`word_list` er en liste af `tokenized` ord (se `word_tokenize`) og `stop_word` er en string bestående af den genre som ordene tilhører.

In [30]:
# indlæs ren, sorteret og med sprog datasettet
df = pd.read_pickle('./data/clean_with_lan.pkl')

In [32]:
# DETTE STEP KAN TAGE NOGET TID

# DETTE STEP KAN TAGE NOGET TID
# serialize to pickle
df.to_pickle('./data/clean_with_lan_and_words.pkl')

### 5. klargør test-og træningssæt til Naive Bayes og træn
Denne del kan kun køres hvis `clean_with_lan_and_words.pkl` datasættet er tilgængeligt. Vær også opmærksom på, at træningen af modellen kan tage noget tid.

Der er fri leg for, at træne en model, for alle eller nogle genre. Bestemt størrelse af `training_set` og `test_test`.

Når en model er træning, find mulighed for at gemme den! Og find passende navn. Evt. et lille dokument med beskrivelse af hvilke sprog den er trænet med, hvor mange tekster den er trænet med og hvor nøjagtig den er.

In [4]:
# udpak zippet datasæt hvis den ikke eksisterer
if not os.path.isfile('./data/clean_with_lan_and_words.pkl'):
    with zipfile.ZipFile('./data/clean_with_lan_and_words.pkl.zip', 'r') as zip_ref:
        zip_ref.extractall('./data/')

# indlæs ren, sorteret og med sprog og ordliste datasættet
df = pd.read_pickle('./data/clean_with_lan_and_words.pkl')


In [68]:
# opret træningssæt (Kun Pop og Rock)
training_set = df[df['genre'] == 'Pop'].words[:1000].tolist() + df[df['genre'] == 'Rock'].words[:1000].tolist()
test_set = df[df['genre'] == 'Pop'].words[1000:1250].tolist() + df[df['genre'] == 'Rock'].words[1000:1250].tolist()

# bland træningssæt
random.shuffle(training_set)
random.shuffle(test_set)

In [70]:
# DETTE STEP KAN TAGE NOGET TID
# Træn Naive Bayes genre-classifieren
classifier = NaiveBayesClassifier.train(training_set)

# Bestem nøjagtigheden, ved brug af test_data
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(f'Accuracy is: , {accuracy * 100}%')


Accuracy is: , 75.0%


In [73]:
# classify en sangtekst
word_list, _ = df[df['genre'] == 'Rock'].reset_index().words[1041]
result = classifier.classify(word_list)
print(result)

Pop
